Foundation API Information
--

In [1]:
# initialize libraries
import requests
from requests import api
from requests.models import Response
from dotenv import load_dotenv, dotenv_values
import json
import bcrypt
from datetime import datetime, timezone
from pprint import pprint
from tabulate import tabulate
import csv
import os

# Load the environment variables from the .env file
load_dotenv()

base_url = os.getenv('BASE_URL')
api_key = os.getenv('API_KEY')
api_key_value = os.getenv('API_KEY_VALUE')


In [2]:
## get_foundation_headers - Use bcrypt to hash the API key with a timestamp per Litera documentation
def get_foundation_headers():
    instant_timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.%fZ")[:-1] + "0Z"  
    combined = api_key_value + "|" + instant_timestamp
    salt = bcrypt.gensalt()
    hashed = bcrypt.hashpw(combined.encode('utf-8'), salt)
    hashed_key = hashed, instant_timestamp
    result = {'x-foundation-api-key': api_key, 'x-foundation-timestamp': hashed_key[1], 'x-foundation-api-auth': hashed_key[0], "x-foundation-impersonate":"JP.Laub@kmob.com", "accept":"application/json"}
    return result

In [ ]:
## Get metadata from the Foundation Metadata API endpoint and return it as JSON
def get_foundation_metadata():
    headers = get_foundation_headers()
    url = f"{base_url}api/v1/application/metadata"
    response = requests.get(url, headers=headers) #, verify=False
    
    if response.status_code == 200:
        metadata = response.json()
        return metadata
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Download the metadata as a Python dictionary
data = json.loads(json.dumps(get_foundation_metadata(), indent=4))

In [ ]:
## Create and export a table of Matter fields with associated configurations
# Construct a table listing matter custom fields with their IDs and descriptions
MCFtable = []
# for field in data['matterCustomFieldTypes']:
#     sourceRecordID = field.get('sourceRecordId', '')
#     field_name = field.get('name', '')
#     field_datatype = field.get('dataType', '')
#     field_description = field.get('description', '')
#     MCFtable.append([sourceRecordID, field_datatype, '', field_name, field_description])

# Append the matter fields to the table
for field in data['matterFields']:
    field_ID = field.get('name', '')
    field_name = field.get('displayName', '')
    # Check for a match between field_ID and the "id" key in matterObjectTypes
    matched_object = next((obj for obj in data.get('matterObjectTypes', []) if obj.get('id') == field_ID), None)
    if matched_object:
        field_ID = matched_object.get('sourceRecordId', field_ID)
    # Check for a match between field_ID and the "id" key in matterCustomFieldTypes
    matched_object = next((obj for obj in data.get('matterCustomFieldTypes', []) if obj.get('id') == field_ID), None)
    if matched_object:
        field_ID = matched_object.get('sourceRecordId', field_ID)
    field_datatype = field.get('dataTypeName', '')
    field_type = field.get('fieldType', '')
    field_description = field.get('description', '')
    # Only append if the field_name is not already present in MCFtable (from matterCustomFieldTypes)
    if field_name not in [row[1] for row in MCFtable]:
        MCFtable.append([field_ID, field_datatype, field_type, field_name, field_description])

# Sort the table by sourceRecordID (first column) in ascending order
MCFtable_sorted = sorted(MCFtable, key=lambda x: x[0])
#print(tabulate(MCFtable_sorted, headers=["Field ID", "Field Data Type", "Field Type", "Field Name", "Description"], tablefmt="github"))

# Export the sorted table to a CSV file
csv_path = r'\\docs-oc\files\KMOBAPPS\Foundation\MetaData\foundation_matter_custom_fields_api.csv'
with open(csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Field ID", "Field Data Type", "Field Type", "Field Name", "Description"])
    writer.writerows(MCFtable_sorted)
print(f"Matter Custom Field CSV exported to {csv_path}")

In [ ]:
## Create and export a table of Person fields with associated configurations

# Construct a table listing people custom fields with their IDs and descriptions
PCFtable = []
for field in data['personCustomFieldTypes']:
    sourceRecordID = field.get('sourceRecordId', '')
    field_name = field.get('name', '')
    field_description = field.get('description', '')
    PCFtable.append([sourceRecordID, field_name, field_description])

# Append the pre-defined person fields to the table
for field in data['personFields']:
    field_ID = field.get('name', '')
    field_name = field.get('displayName', '')
    field_description = field.get('description', '')
    # Only append if the field_name is not already present in PCFtable (from personCustomFieldTypes)
    if field_name not in [row[1] for row in PCFtable]:
        PCFtable.append([field_ID, field_name, field_description])

# Sort the table by sourceRecordID (first column) in ascending order
PCFtable_sorted = sorted(PCFtable, key=lambda x: x[0])
print(tabulate(PCFtable_sorted, headers=["Field ID", "Field Name", "Description"], tablefmt="github"))

# Export the sorted table to a CSV file
# \\docs-oc\files\KMOBAPPS\Foundation\MetaData
csv_path = r'\\docs-oc\files\KMOBAPPS\Foundation\MetaData\foundation_person_custom_fields_api.csv'
with open(csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Field ID", "Field Name", "Description"])
    writer.writerows(PCFtable_sorted)
print(f"Person Custom Field CSV exported to {csv_path}")

In [ ]:
## search_all_attys - Query the Search API endpoint for firm attorneys
def search_all_attys():
    headers = get_foundation_headers()
    url = f"{base_url}search/search"
    #office = input("Enter the office location to search for: ")
    params = {
        "q": f"Person((OR(personProfileType~Attorneys)))",
    #    "facetBehavior": "0",
    #    "specificFacets": "string",
    #    "sort": "string",
        "take": "1000",
        "skip": "0"
    }
    response = requests.get(url, headers=headers, params=params)
    print({response.status_code})

    if response.status_code == 200:
        all_atty_data = response.json()
        #pprint(metadata)
        return all_atty_data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None
    
#search_response = search_people_by_office()

# Download the metadata as a Python dictionary
people_data = json.loads(json.dumps(search_all_attys(), indent=4))

In [ ]:
## search_all_people_by_office - Query the Search API endpoint for firm people by office
def search_all_people_by_office():
    headers = get_foundation_headers()
    url = f"{base_url}search/search"
    office = input("Enter the office location to search for: ")
    params = {
        "q": f"Person((OR(primaryOffice~{office})))",
    #    "facetBehavior": "0",
    #    "specificFacets": "string",
    #    "sort": "string",
        "take": "500",
        "skip": "0"
    }
    response = requests.get(url, headers=headers, params=params)
    print({response.status_code})

    if response.status_code == 200:
        people_by_office_data = response.json()
        #pprint(metadata)
        return people_by_office_data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None
    
#search_response = search_people_by_office()

# Download the metadata as a Python dictionary
people_data = json.loads(json.dumps(search_all_people_by_office(), indent=4))

In [ ]:
## Find and display all attorneys

# Initialize a list to hold the people data by office
people_by_office_table = []

#Construct a table that lists lawyer data for the selected office
for person in people_data['items']:
    person_ID = person.get('id', '')
    person_firstname = person.get('first', '')
    person_lastname = person.get('last', '')
    person_title = person.get('title', '')
    person_office = person.get('primaryOffice', '')
    person_website = person.get('urlWebSite', '')
    if person_website is None:
        people_by_office_table.append([person_ID, person_firstname, person_lastname, person_title, person_office, person_website])

# Sort the table by sourceRecordID (first column) in ascending order
people_by_office_table_sorted = sorted(people_by_office_table, key=lambda x: x[2]) #Sort by last name
print(f"Number of people in table: {len(people_by_office_table_sorted)}")
print(tabulate(people_by_office_table_sorted, headers=["Person ID", "First Name", "Last Name", "Title", "Office", "Website"], tablefmt="github"))



In [10]:
## search_all_pitches_rfps - Query the Search API endpoint for pitch/RFP packets

# Query the Search API endpoint for firm attorneys
def search_all_pitches_rfps():
    headers = get_foundation_headers()
    url = f"{base_url}search/search"
    params = {
    #    "q": f"Packet()",
    #    "q": f"Packet((OR(packetTypeId~870d5871-dbc7-42c1-ae26-f5825d62fef8~Pitch/RFP)))",
        "q": f"Packet((OR(packetTypeId~870d5871-dbc7-42c1-ae26-f5825d62fef8~Pitch/RFP)) (OR(tagCategory-ab5a0c6f-b9d0-401e-8f6d-d8e843ce52ba-DAYRANGE~365)))",
    #    "facetBehavior": "0",
    #    "specificFacets": "string",
    #    "sort": "string",
        "take": "500",
        "skip": "0"
    }
    response = requests.get(url, headers=headers, params=params)
    print({response.status_code})

    if response.status_code == 200:
        all_pitch_rfp_data = response.json()
        pprint(all_pitch_rfp_data)
        return all_pitch_rfp_data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None
    
# Download the metadata as a Python dictionary
pitch_rfp_data = json.loads(json.dumps(search_all_pitches_rfps(), indent=4))
#pprint(pitch_rfp_data)

{200}
{'count': 83,
 'docTypeFacet': None,
 'facets': [],
 'items': [{'attachments': [],
            'canEdit': True,
            'client': None,
            'clientId': None,
            'clientName': None,
            'clientNumber': None,
            'createDate': '2025-05-05T19:54:00',
            'creatorName': 'Fernando Guilarte',
            'customFields': {'195bbaea-9463-48fa-8cff-c86f15e1fc2d': ['1473',
                                                                      '1607',
                                                                      '1609',
                                                                      '1486'],
                             '3a4d4cbe-5bcf-4603-9250-c30eb73c8045': ['Upwork '
                                                                      'Inc., '
                                                                      'formerly '
                                                                      'Elance-oDesk, '
                    

In [13]:
## Find and display all pitch/RFP packets

# Initialize a list to hold the people data by office
pitch_rfp_table = []

#Construct a table that lists lawyer data for the selected office
for packet in pitch_rfp_data['items']:
    packet_ID = packet.get('id', '')
    packet_name = packet.get('name', '')
    custom_fields = packet.get('customFieldsDisplay', {})
    # Get the specific field value
    packet_pitch_status = custom_fields.get('7cfd3225-2a70-4e21-9c56-55924b44c7c9', '')
    packet_pitch_content_matt = custom_fields.get('195bbaea-9463-48fa-8cff-c86f15e1fc2d', '')
    packet_target_type = custom_fields.get('bbbcc750-ddb9-4e50-b422-e384d63a60bd', '')
    packet_gen_prac_scope = custom_fields.get('a5040ad6-e8d0-4a10-bbcb-9588e701ce70', '')
    packet_case_cite = custom_fields.get('45eb3006-061b-4db0-9946-6e5040b5d4bd', '')
    packet_kmob_industry = custom_fields.get('83be4625-df49-4dee-98de-07b68256d9a2', '')
    #packet_title = packet.get('title', '')
    #packet_office = packet.get('primaryOffice', '')
    #packet_website = packet.get('urlWebSite', '')
    pitch_rfp_table.append([packet_ID, packet_name, packet_pitch_status, packet_pitch_content_matt, packet_target_type, packet_gen_prac_scope, packet_case_cite, packet_kmob_industry])

# Sort the table by sourceRecordID (first column) in ascending order
pitch_rfp_table_sorted = sorted(pitch_rfp_table, key=lambda x: x[1]) #Sort by packet name
print(f"Number of packets in table: {len(pitch_rfp_table_sorted)}")
print(tabulate(pitch_rfp_table_sorted, headers=["Packet ID", "Packet Name", "Pitch/RFP Status", "Pitch Content Matter", "Target Type", "General Practice Scope", "Case Citation", "Knobbe Industry"], tablefmt="github"))

Number of packets in table: 83
| Packet ID                            | Packet Name                                                | Pitch/RFP Status   | Pitch Content Matter                                                                                                                                                                                                                                                                                                                                                                 | Target Type         | General Practice Scope                                                                                                                                                                                                                                                                              | Case Citation                                                                                                                          | Knobbe Industry   